In [ ]:
pip install tensorflow==1.15

     |████████████████████████████████| 412.3MB 39kB/s 
     |████████████████████████████████| 3.8MB 48.6MB/s 
     |████████████████████████████████| 51kB 7.6MB/s 
     |████████████████████████████████| 512kB 44.5MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7542 sha256=f0c284329acccd72ec012fff6c2b4f626f7da4611e4a948c5af35f00a282d24e
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tens

In [ ]:
d = 1
if d == 1 :
  !git clone https://github.com/Skuldur/Classical-Piano-Composer
  files_path ='/content/Classical-Piano-Composer/midi_songs'
elif d == 2 :
  !wget https://storage.googleapis.com/magentadata/datasets/maestro/v2.0.0/maestro-v2.0.0-midi.zip
  !unzip maestro-v2.0.0-midi.zip
  files_path = '/content/maestro-v2.0.0/2018'
else:
  !git clone https://github.com/corynguyen19/midi-lstm-gan/
  files_path = ''

Cloning into 'Classical-Piano-Composer'...
remote: Enumerating objects: 334, done.
remote: Total 334 (delta 0), reused 0 (delta 0), pack-reused 334
Receiving objects: 100% (334/334), 721.79 MiB | 17.26 MiB/s, done.
Resolving deltas: 100% (41/41), done.


In [ ]:
import sys
import matplotlib.pyplot as plt
import numpy as np
import pickle
import glob
from tensorflow.compat.v1 import keras

from __future__ import print_function, division
from music21 import converter, instrument, note, chord, stream
from tensorflow.compat.v1.keras.layers import Input, Dense, Reshape, Dropout, CuDNNLSTM, Bidirectional
from tensorflow.compat.v1.keras.layers import BatchNormalization, Activation, ZeroPadding2D
from tensorflow.compat.v1.keras.layers import LeakyReLU
from tensorflow.compat.v1.keras.models import Sequential, Model
from tensorflow.compat.v1.keras.optimizers import Adam
from tensorflow.compat.v1.keras.utils import to_categorical,plot_model
from tensorflow.python.compiler.tensorrt import trt_convert as trt

def get_notes():
    """ Get all the notes and chords from the midi files """
    notes = []

    for file in glob.glob(files_path+"/*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)
        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes
            
        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))
            elif isinstance(element, note.Rest):
                notes.append('SC')    
    with open('GAN_NoteList','wb') as F:
      pickle.dump(notes,F)       
    
    return notes

def prepare_sequences(notes):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100
    n_vocab=len(set(notes))

    # Get all possible notes (pitch, chords, SC)
    noteNames = sorted(set(item for item in notes))

    # Create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(noteNames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length] #the next note in the seq
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input) #how many seq to we have 

    # Reshape the input into a format compatible with LSTM layers
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    
    # Normalize input between -1 and 1
    network_input = (network_input - float(n_vocab)/2) / (float(n_vocab)/2)
    network_output = to_categorical(network_output)

    return (network_input, network_output)

def generate_notes(model, network_input):
    """ Generate notes from the neural network based on a sequence of notes """
        
    n_vocab=len(set(notes))
    

    # pick a random sequence from the input as a starting point for the prediction
    start = numpy.random.randint(0, len(network_input)-1)
    
    # Get pitch names and store in a dictionary
    noteNames = sorted(set(item for item in notes))
    int_to_note = dict((number, note) for number, note in enumerate(noteNames))

    pattern = network_input[start]
    prediction_output = []

    # generate 500 notes
    for note_index in range(500):
        prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
        #prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = numpy.argmax(prediction)
        #ind = np.unravel_index(np.argmax(a, axis=None), a.shape) result=predection[ind] !
        result = int_to_note[index]
        prediction_output.append(result)
        
        pattern = numpy.append(pattern,index)
        #pattern.append(index)
        pattern = pattern[1:len(pattern)] #len(pattern)-1 ?

    return prediction_output

def create_midi(prediction_output, filename):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for item in prediction_output:
        pattern = item[0]
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='{}.mid'.format(filename))

class GAN():
    def __init__(self, rows):
        self.seq_length = rows
        self.seq_shape = (self.seq_length, 1)
        self.latent_dim = 1000
        self.disc_loss = []
        self.gen_loss =[]
        self.notes = get_notes()
        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates note sequences
        z = Input(shape=(self.latent_dim,))
        generated_seq = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        validity = self.discriminator(generated_seq)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_discriminator(self):

        model = Sequential()
        model.add(CuDNNLSTM(512, input_shape=self.seq_shape, return_sequences=True)) #faster then LSTM 
        model.add(Bidirectional(CuDNNLSTM(512)))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid')) #binary_classification
        model.summary()
        plot_model(model,to_file='descriminator.png',show_shapes=True)
        seq = Input(shape=self.seq_shape)
        validity = model(seq)

        return Model(seq, validity)
      
    def build_generator(self):

        model = Sequential()
        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.seq_shape), activation='tanh'))
        model.add(Reshape(self.seq_shape))
        model.summary()
        plot_model(model,to_file='generator.png',show_shapes=True)
        noise = Input(shape=(self.latent_dim,))
        seq = model(noise)

        return Model(noise, seq)

    def train(self, epochs, batch_size=128, sample_interval=50):
        

        X_train, y_train = prepare_sequences(self.notes)

        # Adversarial ground truths
        real = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        
        # Training the model
        for epoch in range(epochs):

            # Training the discriminator
            # Select a random batch of note sequences
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            real_seqs = X_train[idx]

            #noise = np.random.choice(range(484), (batch_size, self.latent_dim))
            #noise = (noise-242)/242
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Generate a batch of new note sequences
            gen_seqs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(real_seqs, real)
            d_loss_fake = self.discriminator.train_on_batch(gen_seqs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake) #la moyenne of the lost fuction of the discriminator


            # Train the generator (to have the discriminator label samples as real)
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            g_loss = self.combined.train_on_batch(noise, real) 

            # Print the progress and save into loss lists
            if epoch % sample_interval == 0:
              print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
              self.disc_loss.append(d_loss[0])
              self.gen_loss.append(g_loss)
        self.generator.save('GAN_model.h5')  

        
        
    def generate(self):
        # Get notes names and store in a dictionary
        generated_tons = []
        base_tons = { 'C':0, 'C#':1, 'D':2, 'D#':3, 'E':4, 'F':5, 'F#':6, 'G':7, 'G#':8, 'A':9, 'A#':10, 'B':11}
   
        noteNames = sorted(set(item for item in self.notes))
        int_to_note = dict((number, note) for number, note in enumerate(noteNames))
        
        # Use random noise to generate sequences
        noise = np.random.normal(0, 1, (1, self.latent_dim))
        predictions = self.generator.predict(noise)
        
        pred_notes = [x*242+242 for x in predictions[0]]
        print(int_to_note)
        print(pred_notes)
        pred_notes = [int_to_note.get(int(x),"0") for x in pred_notes]

        generated_tons = [x for x in pred_notes if (x[0].isalpha() and x!='SC')]
        print(generated_tons)
        print("tone span ="+str(base_tons[max(generated_tons)[0]]+8*int(max(generated_tons)[-1])-base_tons[min(generated_tons)[0]]+8*int(min(generated_tons)[-1])))
        print("unique tones = "+str(len(set(pred_notes))))
        print(pred_notes)
        create_midi(pred_notes, 'GAN_')
    def plot_loss(self):
        plt.plot(self.disc_loss, c='red')
        plt.plot(self.gen_loss, c='blue')
        plt.title("GAN Loss per Epoch")
        plt.legend(['Discriminator', 'Generator'])
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.savefig('GAN_Loss_per_Epoch_final.png', transparent=True)
        plt.close()
if __name__ == '__main__':
  gan = GAN(rows=100)    
  gan.train(epochs=600, batch_size=32, sample_interval=1)
  gan.plot_loss()

Parsing /content/Classical-Piano-Composer/midi_songs/ahead_on_our_way_piano.mid
Parsing /content/Classical-Piano-Composer/midi_songs/gerudo.mid
Parsing /content/Classical-Piano-Composer/midi_songs/8.mid
Parsing /content/Classical-Piano-Composer/midi_songs/DOS.mid
Parsing /content/Classical-Piano-Composer/midi_songs/ff4-airship.mid
Parsing /content/Classical-Piano-Composer/midi_songs/ff4pclov.mid
Parsing /content/Classical-Piano-Composer/midi_songs/Ff4-BattleLust.mid
Parsing /content/Classical-Piano-Composer/midi_songs/Oppressed.mid
Parsing /content/Classical-Piano-Composer/midi_songs/Finalfantasy6fanfarecomplete.mid
Parsing /content/Classical-Piano-Composer/midi_songs/FF4.mid
Parsing /content/Classical-Piano-Composer/midi_songs/FFIXQuMarshP.mid
Parsing /content/Classical-Piano-Composer/midi_songs/Suteki_Da_Ne_(Piano_Version).mid
Parsing /content/Classical-Piano-Composer/midi_songs/Eternal_Harvest.mid
Parsing /content/Classical-Piano-Composer/midi_songs/tifap.mid
Parsing /content/Classi

In [ ]:
gan.generate()

{0: '0', 1: '0.1', 2: '0.1.5', 3: '0.1.6', 4: '0.2', 5: '0.2.3.7', 6: '0.2.4.7', 7: '0.2.5', 8: '0.2.6', 9: '0.2.7', 10: '0.3', 11: '0.3.5', 12: '0.3.5.8', 13: '0.3.6', 14: '0.3.6.8', 15: '0.3.6.9', 16: '0.3.7', 17: '0.4', 18: '0.4.5', 19: '0.4.6', 20: '0.4.7', 21: '0.5', 22: '0.5.6', 23: '0.6', 24: '1', 25: '1.2', 26: '1.2.4.6.8.10', 27: '1.2.6', 28: '1.2.6.8', 29: '1.3', 30: '1.3.5', 31: '1.3.5.8', 32: '1.3.6', 33: '1.3.7', 34: '1.3.8', 35: '1.4', 36: '1.4.6', 37: '1.4.6.9', 38: '1.4.7', 39: '1.4.7.10', 40: '1.4.7.9', 41: '1.4.8', 42: '1.5', 43: '1.5.8', 44: '1.5.9', 45: '1.6', 46: '1.7', 47: '10', 48: '10.0', 49: '10.0.2.5', 50: '10.0.3', 51: '10.0.4', 52: '10.0.5', 53: '10.1', 54: '10.1.3', 55: '10.1.3.5.6', 56: '10.1.3.6', 57: '10.1.4', 58: '10.1.4.6', 59: '10.1.5', 60: '10.11', 61: '10.11.3', 62: '10.11.3.5', 63: '10.2', 64: '10.2.3', 65: '10.2.4', 66: '10.2.5', 67: '10.3', 68: '11', 69: '11.0', 70: '11.0.4', 71: '11.0.4.6', 72: '11.0.4.7', 73: '11.0.5', 74: '11.1', 75: '11.1.4',